# Models subpackage

The `opr.models` subpackage contains ready-to-use neural networks implemented in PyTorch, featuring a common interface.

## Usage example - Place Recognition models

### LiDAR-based models with MinkowskiEngine

In [1]:
import MinkowskiEngine as ME
import torch

from opr.models.place_recognition import MinkLoc3D

model = MinkLoc3D()

# Create a sample batch input dict
batch_size = 2
n_points = 4096
feats_list = [torch.randn(n_points, 1) for _ in range(batch_size)]
coords_list = [
    torch.FloatTensor(n_points, 3).uniform_(-1, 1) for _ in range(batch_size)
]  # sample points in [-1, 1]
quantized_coords_list = []
quantized_feats_list = []
for coords, feats in zip(coords_list, feats_list):
    quantized_coords, quantized_feats = ME.utils.sparse_quantize(
        coordinates=coords,
        features=feats,
        quantization_size=0.01,
    )
    quantized_coords_list.append(quantized_coords)
    quantized_feats_list.append(quantized_feats)
quantized_coords_list = [
    ME.utils.sparse_quantize(coords, quantization_size=0.01) for coords in coords_list
]  # quantize points to [-1, 1] with quantization size 0.01
batch = {
    "pointclouds_lidar_coords": ME.utils.batched_coordinates(quantized_coords_list),
    "pointclouds_lidar_feats":  torch.cat(quantized_feats_list),
}

print(f"batch['pointclouds_lidar_coords'].shape = {batch['pointclouds_lidar_coords'].shape}")
print(f"batch['pointclouds_lidar_feats'].shape = {batch['pointclouds_lidar_feats'].shape}\n")

# Forward pass
output = model(batch)

print(f"type(output): {type(output)}")
print(f"output.keys(): {output.keys()}")
print(f"type(output['final_descriptor']): {type(output['final_descriptor'])}")
print(f"output['final_descriptor'].shape: {output['final_descriptor'].shape}")


batch['pointclouds_lidar_coords'].shape = torch.Size([8189, 4])
batch['pointclouds_lidar_feats'].shape = torch.Size([8189, 1])

type(output): <class 'dict'>
output.keys(): dict_keys(['final_descriptor'])
type(output['final_descriptor']): <class 'torch.Tensor'>
output['final_descriptor'].shape: torch.Size([2, 256])
